In [ ]:
# Import the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pmdarima as pm 

# Time series analysis
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf # type: ignore
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
from pmdarima.arima import auto_arima
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

## 1. Overview of the dataset:
The series that is the subject of this notebook is the monthly average price of "Brent" oil from January 2011 to May 2022.

In [ ]:
dateparse = lambda dates: pd.to_datetime(dates, format='%d/%m/%Y')
dataset = pd.read_csv('./Brent Oil Historical Data.csv', parse_dates=['Date'], index_col='Date', date_parser=dateparse)
dataset.head(10)

We see that our dataset contains several variables (Price, Open, High...), and the observations start from 1988... but we are only interested in the variable Price, which represents the monthly closing price, from January 2011.

In [ ]:
df = pd.DataFrame(dataset)
data = df[['Price']].copy()
data = data['2011':]
data.head()

In [ ]:
print(data.describe())

Above, a summary of the statistics of the dataset.

Now we will see the curve of the price fluctuation;

In [ ]:
data.plot(figsize=(10,5))
plt.show()

## 2. Stationarity analysis:

#### Simple and Partial ACFs:


In [ ]:
plot_acf(data), plot_pacf(data)
plt.show()

#### Time series decomposition

In [10]:
def decomposition_plot(ts):
# Apply seasonal_decompose 
    decomposition = sm.tsa.seasonal_decompose(ts, period=12)
    
# Get trend, seasonality, and residuals
    trend = decomposition.trend
    seasonal = decomposition.seasonal
    residual = decomposition.resid

# Plotting
    plt.figure(figsize=(9,6))
    plt.subplot(411)
    plt.plot(ts, label='Original', color='blue')
    plt.legend(loc='best')
    plt.subplot(412)
    plt.plot(trend, label='Trend', color='blue')
    plt.legend(loc='best')
    plt.subplot(413)
    plt.plot(seasonal,label='Seasonality', color='blue')
    plt.legend(loc='best')
    plt.subplot(414)
    plt.plot(residual, label='Residuals', color='blue')
    plt.legend(loc='best')
    plt.tight_layout()

In [ ]:
decomposition_plot(data)

#### Variation of the mean and variance

In [12]:
def test_stationarity(timeseries):
    #Determing rolling statistics
    plt.figure(figsize=(13,6))
    plt.plot(timeseries, color='blue',label='Original')
    plt.plot(timeseries.rolling(window= 12,center= False).mean(),color='red',label='Moyenne')
    plt.plot(timeseries.rolling(window=12,center= False).std(),color='orange',label='Variance')
    plt.legend()

In [ ]:
test_stationarity(data)

By observing the series graph, it is evident that the series is not stationary, the mean and variance vary over time.

#### Augmented Dickey-Fuller Test:

We define the function Dickey_Fuller(-) which performs the augmented Dickey-Fuller test and returns the results.

In [14]:
# Stationarity tests
def Dickey_Fuller(timeseries):
    print('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print(dfoutput)

In [ ]:
Dickey_Fuller(data)

We observe in the test results that the t-statistic |t| = 1.66 < 3.47 the critical value (1%) with a p-value = 0.45 > 0.05, which means that the model has a unit root, so the studied process is non-stationary of type "DS".

#### Stationarization:

To stationarize this process, we will perform a first-order differencing, which is a very common and efficient method.

In [16]:
diff1 = data.diff()
diff1.dropna(inplace=True)

In [ ]:
test_stationarity(diff1)

We see in the test results that the t-statistic |t| = 9.41 > 3.47 the critical value (1%) with a p-value << 0.05 very negligible, which confirms that our differenced series is indeed stationary.

In [ ]:
plot_acf(diff1)
plot_pacf(diff1)
plt.show()

#### Analysis of Simple and Partial Correlograms:
By visualizing the correlograms of our stationarized series, we note the following:
* The PACF cuts the confidence interval starting from lag p = 3
* The ACF cuts the confidence interval starting from lag q = 3

#### Sampling:

Before proceeding to the selection and adjustment of the model, we will first perform sampling on our dataset by dividing it into a training sample (80%) and a test sample (20%); We will calculate our forecasts on the latter.

In [26]:
# Train Test Split Index
train_size = 0.8
split_id = round(len(diff1)* train_size)

# Split
train = diff1.iloc[:split_id]
test = diff1.iloc[split_id:]
test1 = data.iloc[split_id:]

## 3. Optimal ARIMA Model Selection:
We can rely on the analysis of the simple and partial correlogram to determine the optimal p and q orders for our model. However, the pmdarima package provides the auto-arima() method that does all the work. 
To find the best model, it optimizes for a given information criterion, one of ('aic', 'aicc', 'bic', 'hqic', 'oob'), and returns the ARIMA(p,d,q) that minimizes its value.

In [ ]:
model = auto_arima(train, start_p=1, start_q=1,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=5, max_q=5, # maximum p and q
                      m=1,              # frequency of series
                      d=1,              # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      start_P=0, 
                      D=1, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)

##### --> The optimal model that minimizes the AIC criterion is ARIMA (3,1,0)


#### Estimation of the ARIMA(3,1,0) model:

In [ ]:
model = ARIMA(train, order=(3, 1, 0)) 
model.fit(train)
print(model.summary())


### Model Validation:

According to the model statistics:

#### 1. Significance of the coefficients:
* The coefficients are all significantly different from 0 (p-values < 0.05)

#### 2. Residual analysis:
* The condition of independence of the residuals is satisfied (no correlation) because the p-value of the Ljung-Box test (Prob(Q)) is greater than 0.05, we cannot reject the null hypothesis of independence.
* We can say that the residual distribution is homoscedastic (constant variance) because the p-value of the heteroscedasticity test (Prob(H)) is greater than 0.05.
* For the normality of the residuals, the p-value of the Jarque-Bera test (Prob(JB)) is less than 0.05, so we can reject H0.


In [ ]:
model.plot_diagnostics(figsize=(14,10))
plt.show()

* The standardized graph describes uniform variance (absence of heteroscedasticity).

* The histogram shows that the residuals necessarily follow a normal distribution (normality).

* The correlogram shows no significant correlation between the residuals (absence of autocorrelation).

##### ==> The residuals are white noise

## 4. Out-of-sample Forecasting:

Next, we will readjust our ARIMA(3,1,0) model on the differenced series, and then perform out-of-sample forecasts.

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
model1 = ARIMA(diff1, order=(3, 1, 0))  
results = model1.fit()  
plt.figure(figsize=(12,7))
plt.plot(train)
plt.plot(results.fittedvalues[split_id:], color='red')
plt.show()

Above, we visualize the forecasts of the differenced process. To return to the original series, we need to perform some necessary transformations.

In [22]:
# Cumulative sum :
pred_diff_cumsum = pd.Series(0,index=data.index)
pred_diff_cumsum = results.fittedvalues.cumsum()
# original serie predictions :
pred = pd.Series(data['Price'].iat[0], index=data.index)
pred = pred.add(pred_diff_cumsum,fill_value=0)
predictions = pred[split_id:]

In [ ]:
plt.figure(figsize=(13,6))
plt.plot(data, label='Real')
plt.plot(predictions, label='Predicted')
plt.legend()
plt.show()

## Conclusion:
We notice that the model used on our time series provides results similar to those of the test data, which reflects the good performance of these predictions.